In [11]:
import json
from xml.etree import ElementTree

In [16]:
data = './data/semeval2016/phone_chinese/Chinese_phones_training.xml'
root = ElementTree.parse(data).getroot()

In [17]:
def xml2list(root):
    string_result = []
    for review in root:
        rid = review.attrib['rid']
        sentences = review[0]
        for sentence in sentences:
            sentence_dict = {}
            sid = sentence.attrib['id'].split(':')[-1]
            text = sentence[0].text
            sentence_dict['rid'] = rid
            sentence_dict['sid'] = sid
            sentence_dict['sentence_text'] = text
            sentence_dict['triplet'] = []

            if len(sentence) > 1:
                opinions = sentence[1]
                for opinion in opinions:
                    sentence_dict['triplet'].append(opinion.attrib)
            string_result.append((text, json.dumps(sentence_dict['triplet'])))

    return string_result

In [18]:
result = xml2list(root)

In [19]:
result

[('今天有幸拿到了港版白色iPhone 5真机，', '[]'),
 ('试玩了一下，说说感受吧：', '[]'),
 ('1. 真机尺寸宽度与4/4s保持一致没有变化，', '[]'),
 ('长度多了大概一厘米，', '[]'),
 ('也就是之前所说的多了一排的图标。', '[]'),
 ('2. 真机重量比上一代轻了很多，',
  '[{"category": "PHONE#DESIGN_FEATURES", "polarity": "positive"}]'),
 ('个人感觉跟i9100的重量差不多。', '[]'),
 ('（用惯上一代的朋友可能需要一段时间适应了）', '[]'),
 ('3. 由于目前还没有版的SIM卡，', '[]'),
 ('无法插卡使用，', '[]'),
 ('有购买的朋友要注意了，', '[]'),
 ('并非简单的剪卡就可以用，', '[]'),
 ('而是需要去运营商更换新一代的SIM卡。', '[]'),
 ('4. 屏幕显示效果确实比上一代有进步，',
  '[{"category": "DISPLAY#QUALITY", "polarity": "positive"}]'),
 ('不论是从清晰度还是不同角度的视角，', '[]'),
 ('iPhone 5绝对要更上一层，', '[]'),
 ('我想这也许是相对上一代最有意义的升级了。', '[]'),
 ('5. 新的数据接口更小，',
  '[{"category": "PORTS#CONNECTIVITY", "polarity": "positive"}]'),
 ('比上一代更好用更方便，', '[]'),
 ('使用的过程会有这样的体会。', '[]'),
 ('6. 从简单的几个操作来讲速度比4s要快，',
  '[{"category": "CPU#OPERATION_PERFORMANCE", "polarity": "positive"}]'),
 ('这个不用测试软件也能感受出来，', '[]'),
 ('比如程序的调用以及照片的拍摄和浏览。', '[]'),
 ('不过，', '[]'),
 ('目前水货市场上坑爹的价格，', '[]'),
 ('最好大家可以再观望一下，', '[]'),
 ('不要急着出手。', '[]'),
 (

In [25]:
import pandas as pd

In [27]:
biaozhu = []

for each in result:
    text, triplets = each
    triplets = json.loads(triplets)
    if not triplets:
        biaozhu.append({'text': text, 'target': '', 'category': '', 'polarity': ''})
    else:
        for t in triplets:
            category = t['category']
            polarity = t['polarity']
            biaozhu.append({'text': text, 'target': '', 'category': category, 'polarity': polarity})
        
        
    

In [31]:
df = pd.DataFrame(biaozhu)
df.head()

,text,target,category,polarity
0,今天有幸拿到了港版白色iPhone 5真机，,,,
1,试玩了一下，说说感受吧：,,,
2,1. 真机尺寸宽度与4/4s保持一致没有变化，,,,
3,长度多了大概一厘米，,,,
4,也就是之前所说的多了一排的图标。,,,


In [62]:
cat = [each.split('#')[1] for each in df['category'] if each]

In [64]:
set(cat)

{'CONNECTIVITY',
 'DESIGN_FEATURES',
 'GENERAL',
 'MISCELLANEOUS',
 'OPERATION_PERFORMANCE',
 'PRICE',
 'QUALITY',
 'USABILITY'}

In [67]:
entity_trans_dict = {
    'BATTERY': '电池、电源',
    'CPU': 'CPU、处理器、芯片',
    'DISPLAY': '屏幕、显示',
    'HARDWARE': '硬件',
    'HARD_DISC': '硬盘',
    'KEYBOARD': '键盘',
    'MEMORY': '内存',
    'MULTIMEDIA_DEVICES': '多媒体',
    'OS': '操作系统',
    'PHONE': '手机整体',
    'PORTS': '接口',
    'POWER_SUPPLY': '充电',
    'SOFTWARE': '软件',
    'SUPPORT': '售后服务',
    'WARRANTY': '保障'
}

attr_trans_dict = {
    'CONNECTIVITY': '连通性',
    'DESIGN_FEATURES': '外观设计',
    'GENERAL': '概括',
    'MISCELLANEOUS': '其他',
    'OPERATION_PERFORMANCE': '性能',
    'PRICE': '价格',
    'QUALITY': '质量',
    'USABILITY': '易用性'
}

In [69]:
for each in df.iterrows():
    if 'USABILITY' in each[1]['category']:
        print(each[1])

text                2.说明书没说明，
target                       
category    SUPPORT#USABILITY
polarity             negative
Name: 277, dtype: object
text             操作系统复杂，
target                  
category    OS#USABILITY
polarity        negative
Name: 1117, dtype: object
text               全键盘的按键太难用了，
target                        
category    KEYBOARD#USABILITY
polarity              negative
Name: 2270, dtype: object
text                         4  外放声音有点小，
target                                  
category    MULTIMEDIA_DEVICES#USABILITY
polarity                        negative
Name: 3131, dtype: object
text                              声音比较大，
target                                  
category    MULTIMEDIA_DEVICES#USABILITY
polarity                        positive
Name: 3153, dtype: object
text                        3G上网那确实快。   
target                                  
category    MULTIMEDIA_DEVICES#USABILITY
polarity                        positive
Name: 3228, dtype: object
text     

text                     5、GPS功能从没成功使用过，
target                                  
category    MULTIMEDIA_DEVICES#USABILITY
polarity                        negative
Name: 6077, dtype: object
text            系统有很多问题，
target                  
category    OS#USABILITY
polarity        negative
Name: 6207, dtype: object
text                地图：经常无法定位，
target                        
category    SOFTWARE#USABILITY
polarity              negative
Name: 6247, dtype: object


In [71]:
df.to_csv('./data/semeval2016/phone_chinese/手机标注.csv', index=False, encoding='gb18030')